# Package
* Builds docker containers for all microservices

In [ ]:
import JupyterFormatter
JupyterFormatter.enable_autoformat();

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Train/Alpha.ipynb");

In [ ]:
import ProgressMeter: @showprogress, next!

In [ ]:
function qrun(cmd)
    run(pipeline(cmd, stdout = devnull))
end;

In [ ]:
function copy(file::String, dst::String)
    mkpath(joinpath(dst, dirname(file)))
    cp(file, joinpath(dst, file))
end;

In [ ]:
function fetch_media_lists(basedir::String)
    app = "$basedir/fetch_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/fetch_media_lists", app)
    copy("notebooks/Collect", app)
    copy("environment", app)
end;

In [ ]:
function import_media_lists(basedir::String)
    app = "$basedir/import_media_lists"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/import_media_lists", app)
    copy("notebooks/Preprocess/ImportLists/import_lists.jl", app)
    for m in ["anime", "manga"]
        copy("data/processed_data/$m.csv", app)
        for s in ["mal", "anilist", "kitsu", "animeplanet"]
            copy("data/processed_data/$s.$m.csv", app)
        end
    end
    copy("data/processed_data/training.timestamps.csv", app)
end;

In [ ]:
function bagofwords_jl(basedir::String)
    app = "$basedir/bagofwords_jl"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/bagofwords_jl", app)
    copy("notebooks/Train/Alpha.ipynb", app)
    copy("notebooks/Train/Baseline/get_user_biases.jl", app)
    copy("data/processed_data/status.csv", app)
    copy("data/processed_data/item_counts.csv", app)
    for m in ALL_MEDIUMS
        copy("data/alphas/baseline/v1/streaming/$m/rating/params.jld2", app)
    end
end;

In [ ]:
function nondirectional(basedir::String)
    app = "$basedir/nondirectional"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/nondirectional", app)
    copy("notebooks/Train/Alpha.ipynb", app)
    copy("data/processed_data/status.csv", app)
    copy("data/processed_data/item_counts.csv", app)
    for m in ALL_MEDIUMS
        for x in ["adaptations", "dependencies", "recaps", "related"]
            copy("data/alphas/nondirectional/v1/training/$m/watch/$x", app)
        end
    end
end;

In [ ]:
function bagofwords_py(basedir::String)
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            app = "$basedir/bagofwords_py_$(medium)_$(metric)"
            if ispath(app)
                rm(app; recursive = true)
            end
            cp("notebooks/Microservices/bagofwords_py", app)
            copy("notebooks/Train/BagOfWords/bagofwords.py", app)
            datadir = "data/alphas/bagofwords/v1/streaming/$medium/$metric"
            copy("$datadir/model.pt", app)
            copy("$datadir/config.json", app)
        end
    end
end;

In [ ]:
function compute(basedir::String)
    app = "$basedir/compute"
    if ispath(app)
        rm(app; recursive = true)
    end
    cp("notebooks/Microservices/compute", app)
    copy("notebooks/Train/Alpha.ipynb", app)
    copy("data/processed_data/status.csv", app)
    copy("data/processed_data/item_counts.csv", app)
    for medium in ALL_MEDIUMS
        for metric in ALL_METRICS
            copy("data/alphas/linear/v1/streaming/$medium/$metric/params.jld2", app)
        end
    end
end;

In [ ]:
function package(basedir::String)
    cp("notebooks/Microservices/app", basedir)
    apps = [
        fetch_media_lists,
        import_media_lists,
        bagofwords_jl,
        bagofwords_py,
        nondirectional,
        compute,
    ]
    for app in apps
        app(basedir)
    end
end;

In [ ]:
cd(joinpath(@__DIR__, "../.."))
basedir = "data/backend";

In [ ]:
if ispath(basedir)
    rm(basedir; recursive = true)
end

In [ ]:
package(basedir);

In [ ]:
cd("data/backend")
qrun(`docker system prune -f`)
qrun(`docker build -t rsys/backend .`);